# Exercise 2

Hi, In this exercise we are going to build an Imputation pipeline with the help of the knowledge we just gained in lecture 2 and to make this happen we will use a car price prediction problem dataset. Lets Impute!!

In [ ]:
pip install feature-engine
#Run this cell  before any other cell

In [1]:
import pandas as pd
import numpy as np

# to split the data sets
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.impute import MissingIndicator
from feature_engine.imputation import MeanMedianImputer , CategoricalImputer, ArbitraryNumberImputer, RandomSampleImputer, EndTailImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
def getData():
    cols=['']